In [28]:
from importer import *
from tqdm import tqdm

import cv2

import matplotlib.pyplot as plt
#parser = argparse.ArgumentParser()

# Machine Details
#parser.add_argument('--gpu', type=str, required=True, help='[Required] GPU to use')


# parser.add_argument('--data_dir', type=str, required=True, help='Path to shapenet rendered images')


#parser.add_argument('--exp', type=str, required=True, help='[Required] Path of experiment for loading pre-trained model')
exp = '/home/ubuntu/3Dreconstruction/3d-lmnet/expts/6/lm_img2ae_car_emd'
#parser.add_argument('--load_best', action='store_true', help='load best val model')
load_best = True
# AE Details
#parser.add_argument('--bottleneck', type=int, required=False, default=512, help='latent space size')
bottleneck = 512
# parser.add_argument('--bn_encoder', action='store_true', help='Supply this parameter if you want bn_encoder, otherwise ignore')
#parser.add_argument('--bn_decoder', action='store_true', help='Supply this parameter if you want bn_decoder, otherwise ignore')
bn_decoder = True
#parser.add_argument('--bn_decoder_final', action='store_true', help='Supply this parameter if you want bn_decoder, otherwise ignore')

#parser.add_argument('--eval_set', type=str, help='Choose from train/valid')
eval_set = 'valid'
#parser.add_argument('--image', type=str, help='Choose from train/valid')
image = '00.png'
#parser.add_argument('--txt', type=str, help='Choose from train/valid')

# Other Args
#parser.add_argument('--visualize', action='store_true', help='supply this parameter to visualize')
visualize = True
#FLAGS = parser.parse_args()

print '-='*50

print '-='*50
''
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

BATCH_SIZE = 1

NUM_POINTS = 1024
NUM_VIEWS = 24
HEIGHT = 128
WIDTH = 128
PAD = 35
INPUT = image

from utils.show_3d import show3d_balls
ballradius = 3
#----------------visualize----------------
def get_row_col(num_pic):
    squr = num_pic ** 0.5
    row = round(squr)
    col = row + 1 if squr - row > 0 else row
    return row, col
 
 

def save_conv_img(conv_img):
    feature_maps = np.squeeze(conv_img,axis=0)
    img_num = feature_maps.shape[2]
    all_feature_maps = []
    for i in range(0,img_num):
        single_feature_map = feature_maps[:,:,i]
        all_feature_maps.append(single_feature_map)
        plt.imshow(single_feature_map)
        plt.savefig('feature_{}'.format(i))
        
    sum_feature_map = sum(feature_map for feature_map in all_feature_maps)
    plt.imshow(sum_feature_map)
    plt.savefig("feature_map_sum.png")



if __name__ == '__main__':

	ip_image = cv2.imread(INPUT)[4:-5, 4:-5, :3]
	ip_image = cv2.resize(ip_image,(128,128),interpolation=cv2.INTER_CUBIC)
	ip_image = cv2.cvtColor(ip_image, cv2.COLOR_BGR2RGB)
	batch_ip = []
	batch_ip.append(ip_image)
	batch_ip = np.array(batch_ip)
	img_inp = tf.placeholder(tf.float32, shape=(1,HEIGHT, WIDTH, 3), name='img_inp')
	# Generate Prediction
	with tf.variable_scope('psgn') as scope:
		z_latent_img, fig1, fig2 = image_encoder_se_pure_vis(img_inp,bottleneck)

		out_img = decoder_with_fc_only(z_latent_img, layer_sizes=[512,1024,np.prod([NUM_POINTS, 3])],
			b_norm=bn_decoder,
			b_norm_finish=False,
			verbose=True,
			scope=scope
			)
	reconstr = tf.reshape(out_img, (BATCH_SIZE, NUM_POINTS, 3))
	bneck_size = bottleneck
	with tf.variable_scope('pointnet_ae') as scope:
		z = encoder_with_convs_and_symmetry(in_signal=reconstr, n_filters=[64,128,128,256,bneck_size], 
			filter_sizes=[1],
			strides=[1],
			b_norm=True,
			verbose=True,
			scope=scope
			)
		out = decoder_with_fc_only(z, layer_sizes=[512,1024,np.prod([NUM_POINTS, 3])],
			b_norm=bn_decoder,
			b_norm_finish=False,
			verbose=True,
			scope=scope
			)
		reconstr_img = tf.reshape(out, (BATCH_SIZE, NUM_POINTS, 3))

	# Perform Scaling
	_, reconstr_img_scaled = scale(reconstr_img,reconstr_img)
	#reconstr_img_scaled = reconstr_img

	#visualize
	#visualize_feature_map(fig1)
	
snapshot_folder = join(FLAGS.exp, 'best')

	 # GPU configuration
	#config = tf.ConfigProto()
	#config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:

	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver()
	load_previous_checkpoint(snapshot_folder, saver, sess, is_training=False)
	tflearn.is_training(False, session=sess)
	if eval_set == 'valid':
		conv_img ,_pred_scaled = sess.run([fig1, reconstr_img_scaled],feed_dict={img_inp:batch_ip})
		print conv_img.shape
		save_conv_img(conv_img)
		if visualize:
			for i in xrange(BATCH_SIZE):
				cv2.imshow('', batch_ip[0])
				print 'Displaying Pr scaled icp 1k'
				#print _pred_scaled
				show3d_balls.showpoints(_pred_scaled[i], ballradius=3)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


ValueError: Variable psgn/Conv2D/W already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/ubuntu/anaconda2/envs/3drecon/lib/python2.7/site-packages/tflearn/variables.py", line 65, in variable
    validate_shape=validate_shape)
  File "/home/ubuntu/anaconda2/envs/3drecon/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 181, in func_with_args
    return func(*args, **current_args)
  File "/home/ubuntu/anaconda2/envs/3drecon/lib/python2.7/site-packages/tflearn/layers/conv.py", line 85, in conv_2d
    restore=restore)
